In [ ]:
import pandas as pd
import numpy as np

from rolling_and_plot import *

In [ ]:
data =""

In [ ]:
def get_sec(time_str):
    d, m, s = time_str.split(":")
    d, h = d.split("d")
    d = int(d)
    h = int(h)
    m = int(m)
    s = int(s)
    return d*86400 + h*3600 + m*60 +s

def pulse_soc_compute(data, dis_capacity, chg_capacity, to_csv_pts = False, to_csv_curve = False):
    '''
    str, float[, bool, bool] -> pd.DataFrame, pd.DataFrame[, .csv file, .csv file]
    Precondition data is a path to a .txt file
    
    Parameters:
    `dis_capacity` float
        the capacity of the LFP cell when it is discharging
    `chg_capacity` float
        the capacity of the LFP cell when it is charging
    `to_csv_pts` bool
        to send the extracted points to a csv file
    `to_csv_curve` bool
        to send the estimated points to a csv file
    '''
    data = pd.read_csv(data, skiprows=[0,1,2], sep="\t")
    
    first_idx = data[data["MD"] == "D"].iloc[0].name
    last_discharge = data[data["MD"]=="D"].iloc[-1].name - first_idx
    
    df = data.loc[first_idx-1:,["Test Time", "Current", "Voltage", "MD"]].copy()
    
    df.columns = [col.lower() for col in df.columns]

    df.loc[df["md"] == "D","current"] *= -1
    
    df.loc[:,"test time (sec)"] = [get_sec(i) for i in df["test time"]]
    mask = df["test time (sec)"].diff()
    mask.iloc[0], mask.iloc[-1] = 1.0, 1.0
    df["delta t"] = mask

    delta_cap = df["current"] / 3600 * df["delta t"]
    
    dis_soc = list((dis_capacity + delta_cap.loc[:last_discharge].cumsum()) / dis_capacity * 100)
    chg_soc = list((delta_cap.loc[last_discharge + 1:].cumsum()) / chg_capacity * 100)
    dis_soc.extend(chg_soc)
    
    df["soc"] = dis_soc
    
#     if to_csv_pts:
#         name = input("Name of output file for SOC-OCV points: ")
#         data.to_csv(name)
#         print( f"\n{name} was created.\n" )
#     if to_csv_curve:
#         name = input("Name of output file for SOC-OCV curve: ")
#         curve.to_csv(name)
#         print( f"\n{name} was created.\n" )
    
    return df[["test time (sec)","delta t","current","voltage","soc"]]

In [ ]:
df = pulse_soc_compute(data, 18.251, 18.257)
df.to_csv("",
         index = False)

In [ ]:
data_plot(data = [df],
          x = ["test time (sec)"],
          y = [ "soc" ],
          title = "SOC vs. Time",
          x_title = "Time (sec)",
          y_title = "SOC (%)",
          mode = "lines",
          color = "darkorchid",
          name = None,
          size = None
         )